In [4]:
# 1. Install everything
!git clone https://github.com/lm-sys/FastChat.git
%cd FastChat
!pip install -e .
!pip install gradio==4.29.0 torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

In [1]:
# 2. Download model (example: flan-t5-small)
!git lfs install
!git clone https://huggingface.co/google/flan-t5-small ./flan-t5-small

Git LFS initialized.
Cloning into './flan-t5-small'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 62 (delta 25), reused 21 (delta 21), pack-reused 34 (from 1)
Unpacking objects: 100% (62/62), 615.41 KiB | 4.59 MiB/s, done.
Filtering content: 100% (5/5), 1.27 GiB | 66.35 MiB/s, done.


In [1]:
# 🚀 1) Controller
!nohup python3 -m fastchat.serve.controller \
  --host 0.0.0.0 \
  --port 21001 > controller.log 2>&1 &

In [2]:
!tail -n 20 controller.log

2025-09-09 12:27:47 | INFO | controller | args: Namespace(host='0.0.0.0', port=21001, dispatch_method='shortest_queue', ssl=False)
2025-09-09 12:27:47 | ERROR | stderr | INFO:     Started server process [1111]
2025-09-09 12:27:47 | ERROR | stderr | INFO:     Waiting for application startup.
2025-09-09 12:27:47 | ERROR | stderr | INFO:     Application startup complete.
2025-09-09 12:27:47 | ERROR | stderr | INFO:     Uvicorn running on http://0.0.0.0:21001 (Press CTRL+C to quit)


In [3]:
# 🚀 2) Model Worker (flan-t5-small)
!nohup python3 -m fastchat.serve.model_worker \
  --model-path ./flan-t5-small \
  --device cuda --num-gpus 1 \
  --controller-address http://localhost:21001 > worker.log 2>&1 &

In [12]:
!tail -n 20 worker.log

E0000 00:00:1757420964.695398    1466 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757420964.701471    1466 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757420964.716740    1466 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757420964.716764    1466 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757420964.716766    1466 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757420964.716769    1466 computation_placer.cc:177] computation placer already registered. Please check linka

In [13]:
# 🚀 3) Gradio Web UI (هيديك لينك public تلقائي)
!nohup python3 -m fastchat.serve.gradio_web_server \
  --controller-url http://localhost:21001 \
  --share > gradio.log 2>&1 &

In [17]:
!tail -n 20 gradio.log

2025-09-09 12:30:23.838072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757421023.859378    1763 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757421023.865305    1763 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757421023.879970    1763 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757421023.879996    1763 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757421023.879999    1763 computation_placer.cc:177] computation placer alr

In [18]:
# 🚀 4) OpenAI-Compatible API Server
!nohup python3 -m fastchat.serve.openai_api_server \
  --host 0.0.0.0 \
  --port 8000 \
  --controller-address http://localhost:21001 > api_server.log 2>&1 &

In [19]:
!tail -n 20 api_server.log

2025-09-09 12:30:51 | INFO | openai_api_server | args: Namespace(host='0.0.0.0', port=8000, controller_address='http://localhost:21001', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_keys=None, ssl=False)
2025-09-09 12:30:51 | ERROR | stderr | INFO:     Started server process [1931]
2025-09-09 12:30:51 | ERROR | stderr | INFO:     Waiting for application startup.
2025-09-09 12:30:51 | ERROR | stderr | INFO:     Application startup complete.
2025-09-09 12:30:51 | ERROR | stderr | INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [28]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8000/v1", api_key="EMPTY")

resp = client.chat.completions.create(
    model="flan-t5-small",
    messages=[{"role": "user", "content": "What is AI ?"}],
)

print("Response:", resp.choices[0].message.content)

Response: AI is the name of an artificial intelligence assistant.
